In [2]:
! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import os
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV
import pickle
import optuna
from xgboost import XGBClassifier

# Register API for Financial Modeling Prep (Financial Statements and Company Fundamentals)
# https://site.financialmodelingprep.com/developer/
# Register API for Federal Reserve Economic Data (For Macroeconomics Data)
# https://fred.stlouisfed.org/docs/api/fred/
# Yahoo Finance does not need an API

warnings.filterwarnings('ignore')

In [10]:
X_test_transformed = pd.read_csv('X_test_transformed.csv')
y_test = pd.read_csv('y_test.csv')
X_train_oversampled = pd.read_csv('X_train_oversampled.csv')
y_train_oversampled = pd.read_csv('y_train_oversampled.csv')
X_validate_transformed = pd.read_csv('X_validate_transformed.csv')
y_validate = pd.read_csv('y_validate.csv')

In [9]:
# testing if file read correctly
X_train_oversampled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7236 entries, 0 to 7235
Data columns (total 92 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   industry                                              7236 non-null   float64
 1   sector                                                7236 non-null   float64
 2   symbol                                                7236 non-null   float64
 3   year                                                  7236 non-null   float64
 4   adjDividend                                           7236 non-null   float64
 5   dps_growth                                            7236 non-null   float64
 6   dps_growth_rate                                       7236 non-null   float64
 7   interestRate                                          7236 non-null   float64
 8   interestRate_percentage_change                        7236

In [15]:
# using only top 8 features
cols = ['dps_growth_rate','dps_growth','interestRate','year','dividendYield','interestRate_percentage_change','freeCashFlowPerShare','adjDividend']
X_train_oversampled = X_train_oversampled[cols]
X_validate_transformed = X_validate_transformed[cols]
X_test_transformed = X_test_transformed[cols]
X_test_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dps_growth_rate                 479 non-null    float64
 1   dps_growth                      479 non-null    float64
 2   interestRate                    479 non-null    float64
 3   year                            479 non-null    float64
 4   dividendYield                   479 non-null    float64
 5   interestRate_percentage_change  479 non-null    float64
 6   freeCashFlowPerShare            479 non-null    float64
 7   adjDividend                     479 non-null    float64
dtypes: float64(8)
memory usage: 30.1 KB


In [16]:
# Logistic Regression
def objective_function(trial):
    C = trial.suggest_float('C', 0.1, 10, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver='liblinear',
        n_jobs=-1
    )
    metrics = ['roc_auc','accuracy','precison','recall','f1-score']
    # Using cross_val_score to get the average precision score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, C: {C}, penalty: {penalty}, ROC-AUC: {roc_auc}")
    return roc_auc


study_lr = optuna.create_study(direction="maximize")
study_lr.optimize(objective_function, n_trials=100)

best_params_lr = study_lr.best_params
print("Best Parameters: ", best_params_lr)
print("Best ROC-AUC Score: ", study_lr.best_value)

# Create and save model
best_model_lr = LogisticRegression(**best_params_lr, solver='liblinear', n_jobs=-1)
with open('best_models_lr_8_features.pkl', 'wb') as file:
    pickle.dump(best_model_lr, file)

[I 2024-05-11 03:06:33,488] A new study created in memory with name: no-name-fbbe8bb7-2c12-41b1-b78c-23cafa17583e
[I 2024-05-11 03:06:33,987] Trial 0 finished with value: 0.7204457786036875 and parameters: {'C': 0.789404277723618, 'penalty': 'l2'}. Best is trial 0 with value: 0.7204457786036875.


Trial 0, C: 0.789404277723618, penalty: l2, ROC-AUC: 0.7204457786036875


[I 2024-05-11 03:06:35,772] Trial 1 finished with value: 0.7465415309494731 and parameters: {'C': 6.61404623532381, 'penalty': 'l1'}. Best is trial 1 with value: 0.7465415309494731.


Trial 1, C: 6.61404623532381, penalty: l1, ROC-AUC: 0.7465415309494731


[I 2024-05-11 03:06:36,297] Trial 2 finished with value: 0.746483488721261 and parameters: {'C': 5.793508394876492, 'penalty': 'l1'}. Best is trial 1 with value: 0.7465415309494731.


...

Trial 97, C: 6.594769192617902, penalty: l1, ROC-AUC: 0.7465522275773793


[I 2024-05-11 03:09:24,938] Trial 98 finished with value: 0.7465969218451771 and parameters: {'C': 7.5908446548909945, 'penalty': 'l1'}. Best is trial 76 with value: 0.7466721592101528.


Trial 98, C: 7.5908446548909945, penalty: l1, ROC-AUC: 0.7465969218451771


[I 2024-05-11 03:09:27,016] Trial 99 finished with value: 0.7466370274712245 and parameters: {'C': 8.919622767153033, 'penalty': 'l1'}. Best is trial 76 with value: 0.7466721592101528.


Trial 99, C: 8.919622767153033, penalty: l1, ROC-AUC: 0.7466370274712245
Best Parameters:  {'C': 9.923593637499346, 'penalty': 'l1'}
Best ROC-AUC Score:  0.7466721592101528


In [17]:
# Decision Tree
def objective_function(trial):
    max_depth = trial.suggest_int('max_depth', 1, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion
    )
    metrics = ['roc_auc','accuracy','precison','recall','f1-score']
    # Using cross_val_score to get the average precision score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, max_depth: {max_depth}, min_samples_split: {min_samples_split}, "
          f"min_samples_leaf: {min_samples_leaf}, criterion: {criterion}, ROC-AUC: {roc_auc}")
    return roc_auc

study_dt = optuna.create_study(direction="maximize")
study_dt.optimize(objective_function, n_trials=100)

best_params_dt = study_dt.best_params
print("Best Parameters: ", best_params_dt)
print("Best ROC-AUC Score: ", study_dt.best_value)

# Create and save model
best_model_dt = DecisionTreeClassifier(**best_params_dt)
with open('best_models_dt_8_features.pkl', 'wb') as file:
    pickle.dump(best_model_dt, file)

[I 2024-05-11 03:17:30,877] A new study created in memory with name: no-name-ff53c9c5-ca7a-4c12-8e10-dc579f5d07cf
[I 2024-05-11 03:17:31,012] Trial 0 finished with value: 0.6200912404575778 and parameters: {'max_depth': 1, 'min_samples_split': 6, 'min_samples_leaf': 6, 'criterion': 'entropy'}. Best is trial 0 with value: 0.6200912404575778.


Trial 0, max_depth: 1, min_samples_split: 6, min_samples_leaf: 6, criterion: entropy, ROC-AUC: 0.6200912404575778


[I 2024-05-11 03:17:31,294] Trial 1 finished with value: 0.9468523160402704 and parameters: {'max_depth': 8, 'min_samples_split': 7, 'min_samples_leaf': 5, 'criterion': 'entropy'}. Best is trial 1 with value: 0.9468523160402704.


Trial 1, max_depth: 8, min_samples_split: 7, min_samples_leaf: 5, criterion: entropy, ROC-AUC: 0.9468523160402704


[I 2024-05-11 03:17:31,551] Trial 2 finished with value: 0.9577235572012113 and parameters: {'max_depth': 8, 'min_samples_split': 6, 'min_samples_leaf': 9, 'criterion': 'gini'}. Best is trial 2 with value: 0.9577235572012113.


...

Trial 97, max_depth: 9, min_samples_split: 6, min_samples_leaf: 9, criterion: gini, ROC-AUC: 0.96138304834042


[I 2024-05-11 03:18:00,104] Trial 98 finished with value: 0.9636035366770267 and parameters: {'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 10, 'criterion': 'gini'}. Best is trial 71 with value: 0.9647682244596147.


Trial 98, max_depth: 10, min_samples_split: 8, min_samples_leaf: 10, criterion: gini, ROC-AUC: 0.9636035366770267


[I 2024-05-11 03:18:00,380] Trial 99 finished with value: 0.9611161229070782 and parameters: {'max_depth': 9, 'min_samples_split': 5, 'min_samples_leaf': 10, 'criterion': 'gini'}. Best is trial 71 with value: 0.9647682244596147.


Trial 99, max_depth: 9, min_samples_split: 5, min_samples_leaf: 10, criterion: gini, ROC-AUC: 0.9611161229070782
Best Parameters:  {'max_depth': 10, 'min_samples_split': 7, 'min_samples_leaf': 10, 'criterion': 'gini'}
Best ROC-AUC Score:  0.9647682244596147


In [18]:
# KNN
def objective_function(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 1, 10)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    p = trial.suggest_int('p', 1, 5)
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])

    model = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights=weights,
        p=p,
        metric=metric
    )
    metrics = ['roc_auc','accuracy','precison','recall','f1-score']
    # Using cross_val_score to get the average precision score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, n_neighbors: {n_neighbors}, weights: {weights}, p: {p}, metric: {metric}, "
          f"ROC-AUC: {roc_auc}")
    return roc_auc


study_knn = optuna.create_study(direction="maximize")
study_knn.optimize(objective_function, n_trials=100)

best_params_knn = study_knn.best_params
print("Best Parameters: ", best_params_knn)
print("Best ROC-AUC Score: ", study_knn.best_value)

# Create and save model
best_model_knn = KNeighborsClassifier(**best_params_knn)
with open('best_models_knn_8_features.pkl', 'wb') as file:
    pickle.dump(best_model_knn, file)

[I 2024-05-11 03:19:24,019] A new study created in memory with name: no-name-750593a6-6824-4293-8b69-9fd890bc820b
[I 2024-05-11 03:19:24,585] Trial 0 finished with value: 0.925883568432222 and parameters: {'n_neighbors': 8, 'weights': 'distance', 'p': 5, 'metric': 'minkowski'}. Best is trial 0 with value: 0.925883568432222.


Trial 0, n_neighbors: 8, weights: distance, p: 5, metric: minkowski, ROC-AUC: 0.925883568432222


[I 2024-05-11 03:19:24,818] Trial 1 finished with value: 0.9472897043692482 and parameters: {'n_neighbors': 7, 'weights': 'distance', 'p': 5, 'metric': 'manhattan'}. Best is trial 1 with value: 0.9472897043692482.


Trial 1, n_neighbors: 7, weights: distance, p: 5, metric: manhattan, ROC-AUC: 0.9472897043692482


[I 2024-05-11 03:19:25,076] Trial 2 finished with value: 0.9479818187828585 and parameters: {'n_neighbors': 9, 'weights': 'distance', 'p': 4, 'metric': 'manhattan'}. Best is trial 2 with value: 0.9479818187828585.


...

Trial 97, n_neighbors: 10, weights: distance, p: 5, metric: manhattan, ROC-AUC: 0.9482379985090897


[I 2024-05-11 03:19:54,728] Trial 98 finished with value: 0.9482379985090897 and parameters: {'n_neighbors': 10, 'weights': 'distance', 'p': 3, 'metric': 'manhattan'}. Best is trial 13 with value: 0.9482379985090897.


Trial 98, n_neighbors: 10, weights: distance, p: 3, metric: manhattan, ROC-AUC: 0.9482379985090897


[I 2024-05-11 03:19:54,985] Trial 99 finished with value: 0.9479818187828585 and parameters: {'n_neighbors': 9, 'weights': 'distance', 'p': 5, 'metric': 'manhattan'}. Best is trial 13 with value: 0.9482379985090897.


Trial 99, n_neighbors: 9, weights: distance, p: 5, metric: manhattan, ROC-AUC: 0.9479818187828585
Best Parameters:  {'n_neighbors': 10, 'weights': 'distance', 'p': 5, 'metric': 'manhattan'}
Best ROC-AUC Score:  0.9482379985090897


In [19]:
# Random Forest
def objective_function(trial):
    n_estimators = trial.suggest_int('n_estimators', 2, 50)
    max_depth = trial.suggest_int('max_depth', 1, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        n_jobs=-1
    )

    # Using cross_val_score to get the average ROC-AUC score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, n_estimators: {n_estimators}, max_depth: {max_depth}, "
          f"min_samples_split: {min_samples_split}, min_samples_leaf: {min_samples_leaf}, ROC-AUC: {roc_auc}")
    return roc_auc


study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(objective_function, n_trials=100)

best_params_rf = study_rf.best_params
print("Best Parameters: ", best_params_rf)
print("Best ROC-AUC: Score: ", study_rf.best_value)

# Create and save model
best_model_rf = RandomForestClassifier(**best_params_rf, n_jobs=-1)
with open('best_models_rf_8_features.pkl', 'wb') as file:
    pickle.dump(best_model_rf, file)

[I 2024-05-11 03:24:11,941] A new study created in memory with name: no-name-f1185cef-cd23-4d83-a567-d566b4c7f0c3
[I 2024-05-11 03:24:12,248] Trial 0 finished with value: 0.9713800637970552 and parameters: {'n_estimators': 3, 'max_depth': 9, 'min_samples_split': 9, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.9713800637970552.


Trial 0, n_estimators: 3, max_depth: 9, min_samples_split: 9, min_samples_leaf: 5, ROC-AUC: 0.9713800637970552


[I 2024-05-11 03:24:13,294] Trial 1 finished with value: 0.98320204197566 and parameters: {'n_estimators': 29, 'max_depth': 8, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.98320204197566.


Trial 1, n_estimators: 29, max_depth: 8, min_samples_split: 4, min_samples_leaf: 2, ROC-AUC: 0.98320204197566


[I 2024-05-11 03:24:13,872] Trial 2 finished with value: 0.8968586345920452 and parameters: {'n_estimators': 23, 'max_depth': 2, 'min_samples_split': 8, 'min_samples_leaf': 10}. Best is trial 1 with value: 0.98320204197566.


...

Trial 97, n_estimators: 48, max_depth: 9, min_samples_split: 8, min_samples_leaf: 2, ROC-AUC: 0.9857730880293477


[I 2024-05-11 03:26:27,201] Trial 98 finished with value: 0.9797426240605132 and parameters: {'n_estimators': 5, 'max_depth': 10, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 94 with value: 0.9883812171161415.


Trial 98, n_estimators: 5, max_depth: 10, min_samples_split: 6, min_samples_leaf: 1, ROC-AUC: 0.9797426240605132


[I 2024-05-11 03:26:28,306] Trial 99 finished with value: 0.9269621393623115 and parameters: {'n_estimators': 46, 'max_depth': 3, 'min_samples_split': 7, 'min_samples_leaf': 3}. Best is trial 94 with value: 0.9883812171161415.


Trial 99, n_estimators: 46, max_depth: 3, min_samples_split: 7, min_samples_leaf: 3, ROC-AUC: 0.9269621393623115
Best Parameters:  {'n_estimators': 45, 'max_depth': 10, 'min_samples_split': 7, 'min_samples_leaf': 3}
Best ROC-AUC: Score:  0.9883812171161415


In [22]:
label_encoder = LabelEncoder()
# Fit the encoder and transform the target variable
y_train_oversampled_encoded = label_encoder.fit_transform(y_train_oversampled)
# This suppresses printing logs
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective_function(trial):
    n_estimators = trial.suggest_int('n_estimators', 2, 50)
    max_depth = trial.suggest_int('max_depth', 1, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.9, log=True)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    gamma = trial.suggest_float('gamma', 0, 1.0)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)

    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        min_child_weight=min_child_weight,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma=gamma,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        use_label_encoder=False,
        n_jobs=-1
    )

    # Using cross_val_score to get the average ROC-AUC score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled_encoded, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, n_estimators: {n_estimators}, max_depth: {max_depth}, learning_rate: {learning_rate},"
          f"min_child_weight: {min_child_weight}, subsample: {subsample}, colsample_bytree: {colsample_bytree}, "
          f"gamma: {gamma}, reg_alpha: {reg_alpha}, reg_lambda: {reg_lambda}, ROC-AUC: {roc_auc}")
    return roc_auc


study_xgb = optuna.create_study(direction="maximize")
study_xgb.optimize(objective_function, n_trials=100)

best_params_xgb = study_xgb.best_params
print("Best Parameters: ", best_params_xgb)
print("Best ROC-AUC Score: ", study_xgb.best_value)

best_model_xgb = XGBClassifier(**best_params_xgb, use_label_encoder=False, n_jobs=-1)
with open('best_models_xgb_8_features.pkl', 'wb') as file:
    pickle.dump(best_model_xgb, file)

Trial 0, n_estimators: 10, max_depth: 3, learning_rate: 0.7897556921350515,min_child_weight: 1, subsample: 0.8863322946209868, colsample_bytree: 0.9013839074424221, gamma: 0.16368015196267438, reg_alpha: 0.7528856955143486, reg_lambda: 0.49558268950776585, ROC-AUC: 0.9837970001460346
Trial 1, n_estimators: 30, max_depth: 1, learning_rate: 0.0027131225788462466,min_child_weight: 6, subsample: 0.7207044600936032, colsample_bytree: 0.6370567279382376, gamma: 0.7058803684540864, reg_alpha: 0.844160657477311, reg_lambda: 0.3624498280654168, ROC-AUC: 0.8332608288356728
Trial 2, n_estimators: 32, max_depth: 4, learning_rate: 0.14012760305789973,min_child_weight: 7, subsample: 0.9813525335168373, colsample_bytree: 0.902782211148908, gamma: 0.448260737770909, reg_alpha: 0.24079831216831704, reg_lambda: 0.7306282963917448, ROC-AUC: 0.9833692561446004
Trial 3, n_estimators: 41, max_depth: 10, learning_rate: 0.15722811333219644,min_child_weight: 10, subsample: 0.9033332053013357, colsample_bytree:

In [23]:
# Model selection - Compare Performance
with open('best_models_lr_8_features.pkl', 'rb') as file:
    best_model_lr = pickle.load(file)
with open('best_models_dt_8_features.pkl', 'rb') as file:
    best_model_dt = pickle.load(file)
with open('best_models_knn_8_features.pkl', 'rb') as file:
    best_model_knn = pickle.load(file)
with open('best_models_rf_8_features.pkl', 'rb') as file:
    best_model_rf = pickle.load(file)
with open('best_models_xgb_8_features.pkl', 'rb') as file:
    best_model_xgb = pickle.load(file)

print("Testing Performances...Please wait")
best_model_lr.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_lr.predict_proba(X_test_transformed)[:, 1]
lr_performance = roc_auc_score(y_test, predicted_probs)

best_model_dt.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_dt.predict_proba(X_test_transformed)[:, 1]
dt_performance = roc_auc_score(y_test, predicted_probs)

best_model_knn.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_knn.predict_proba(X_test_transformed)[:, 1]
knn_performance = roc_auc_score(y_test, predicted_probs)

best_model_rf.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_rf.predict_proba(X_test_transformed)[:, 1]
rf_performance = roc_auc_score(y_test, predicted_probs)

best_model_xgb.fit(X_train_oversampled, y_train_oversampled_encoded)
predicted_probs = best_model_xgb.predict_proba(X_test_transformed)[:, 1]
xgb_performance = roc_auc_score(y_test, predicted_probs)

# Test performance of the models are
print(f"Logistic Regression Test ROCAUC: {lr_performance}")
print(f"Decision Tree Test ROCAUC: {dt_performance}")
print(f"KNN Test ROCAUC: {knn_performance}")
print(f"Random Forest Test ROCAUC: {rf_performance}")
print(f"XGBoost Test ROCAUC: {xgb_performance}")

Testing Performances...Please wait
Logistic Regression Test ROCAUC: 0.7764652014652015
Decision Tree Test ROCAUC: 0.5483974358974358
KNN Test ROCAUC: 0.7525641025641026
Random Forest Test ROCAUC: 0.7332417582417582
XGBoost Test ROCAUC: 0.7761904761904761
